In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction = 0.333)
sess = tf.compat.v1.Session(config = tf.compat.v1.ConfigProto(gpu_options = gpu_options))

X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

X = X/255.0
y = np.array(y)

dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = '{}-conv-{}-nodes-{}-dense-{}'.format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir = 'logs/{}'.format(NAME))
            
            model = Sequential()
            
            model.add(Conv2D(layer_size, (3, 3), input_shape = X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size = (2, 2)))
            
            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size = (2, 2)))

            model.add(Flatten()) #2D -> 1D
            
            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))
                
            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            model.compile(loss = "binary_crossentropy", optimizer = "adam", 
                          metrics = ['accuracy'])

            model.fit(X, y, batch_size = 32, epochs = 10, 
                      validation_split = 0.1,
                      callbacks = [tensorboard])

Epoch 1/10
702/702 [==============================] - 15s 21ms/step - loss: 0.6225 - accuracy: 0.6513 - val_loss: 0.5915 - val_accuracy: 0.6894
Epoch 2/10
702/702 [==============================] - 16s 23ms/step - loss: 0.5553 - accuracy: 0.7191 - val_loss: 0.5507 - val_accuracy: 0.7234
Epoch 3/10
702/702 [==============================] - 17s 25ms/step - loss: 0.5206 - accuracy: 0.7473 - val_loss: 0.5316 - val_accuracy: 0.7499
Epoch 4/10
702/702 [==============================] - 16s 22ms/step - loss: 0.4992 - accuracy: 0.7607 - val_loss: 0.5247 - val_accuracy: 0.7447
Epoch 5/10
702/702 [==============================] - 16s 22ms/step - loss: 0.4819 - accuracy: 0.7717 - val_loss: 0.5194 - val_accuracy: 0.7503
Epoch 6/10
702/702 [==============================] - 16s 23ms/step - loss: 0.4671 - accuracy: 0.7811 - val_loss: 0.5238 - val_accuracy: 0.7479
Epoch 7/10
702/702 [==============================] - 15s 22ms/step - loss: 0.4525 - accuracy: 0.7934 - val_loss: 0.5222 - val_accuracy:

KeyboardInterrupt: 